In [14]:
from PIL import Image
import pandas as pd
import numpy as np
from glob import glob
from os import path
import random 
import matplotlib.pyplot as plt 
import tensorflow
import imageio
from scipy import signal
from pathlib import Path
from sklearn.model_selection import train_test_split


In [7]:
## IMPORTS ALL IMAGES WITHIN THE FOLDER "imager_dir" INTO TWO ARRAYS, ONE FOR GAUSS POINTS AND ONE FOR IMAGE CROPS ##
all_image_array=np.zeros((256,256))[None, :, :]
all_image_array_gauss=np.zeros((256,256))[None, :, :]

files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_bf_pos'
joined_path = os.path.join(files_dir, images_dir)

for image_file in os.listdir(joined_path):
    image, date, cell_type, bf_fl, index, number_gauss  = image_file.split('_')
    joined_image_path = os.path.join(files_dir, images_dir, image_file)

    if 'gauss' in number_gauss:
        img_gauss = Image.open(joined_image_path)
        image_array_gauss = np.zeros((img_gauss.n_frames,256,256))
        for i in range(img_gauss.n_frames-1):
            img_gauss.seek(i)
            image_array_gauss[i,:,:] = np.array(img_gauss)
        all_image_array_gauss = np.concatenate([all_image_array_gauss, image_array_gauss])
    else:
        img = Image.open(joined_image_path)
        image_array = np.zeros((img.n_frames,256,256))
        for i in range(img.n_frames-1):
            img.seek(i)
            image_array[i,:,:] = np.array(img)
        all_image_array = np.concatenate([all_image_array, image_array])


In [ ]:
## DATA SPLIT INTO VALIDATION AND AUGMENTATION ##

data_set_test_trainvalid_ratio = 0.1
data_split_state = None   # Random split on each call
augmentation_data, validation_data, augmentation_data_gauss, validation_data_gauss =  train_test_split(all_image_array, all_image_array_gauss, 
                                                                                                       test_size=data_set_test_trainvalid_ratio, random_state=data_split_state)

In [23]:
## ANYTHING BELOW THIS POINT IS EDITS ON MITOSPLITNET CODE ##
from mitosplit_net import preprocessing, augmentation, evaluation, training, plotting, util
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import label2rgb
from skimage import segmentation
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, ElasticTransform, GaussNoise, RandomCrop, Resize

In [33]:
## AUGMENTATION OF DATA (I need to go through this again) ##

def augImg(input_img, output_img, transform, **kwargs):
    input_mask = (input_img>0).astype(np.uint8)
    transformed = transform(image=input_img, image0=output_img)
    
    aug_input_img, aug_output_img= transformed['image'], transformed['image0']
    
    # aug_fission_coords = preprocessing.fissionCoords(aug_labels, aug_output_img)
    # aug_output_img, aug_fission_props = preprocessing.prepareProc(aug_output_img, coords=aug_fission_coords, **kwargs)
    return aug_input_img.astype(np.uint8), aug_output_img
    

def augStack(input_data, output_data, transform, **kwargs):
    aug_input_data = np.zeros(input_data.shape, dtype=np.uint8)
    aug_output_data = np.zeros(output_data.shape, dtype=np.float32)
    
    for i in tqdm(range(input_data.shape[0]), total=input_data.shape[0]):
        aug_input_data[i], aug_output_data[i]= augImg(input_data[i], output_data[i], transform, **kwargs)
    return aug_input_data, aug_output_data

In [39]:
img_size = 256
transform = Compose([RandomRotate90(p=0.5), HorizontalFlip(p=0.5), Flip(p=0.5)])

aug_data, aug_data_gauss= augStack(augmentation_data, augmentation_data_gauss,transform, sigma=8)


100%|██████████| 520/520 [00:00<00:00, 1472.88it/s]


(520, 256, 256)